<a href="https://colab.research.google.com/github/hapava-angel/school_x_python_learning/blob/master/transf%2B%22gibrig%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np

In [21]:
%pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.5 MB/s eta 0:00:00


In [22]:
import accelerate

In [23]:
%pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 31.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
import torch.nn.functional as F

In [5]:
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00


In [26]:
from datasets import Dataset
from datasets import load_dataset

In [27]:
dataset = load_dataset('daily_dialog')
#пришлось загружать dataset так, потому что на kaggle в нём всего 996 строк, а здесь даже validation и test  больше

Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [28]:
dataset

DatasetDict({
    train: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 11118
    })
    validation: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
})

In [29]:
def concatenate_utterances(example):
    example['dialog'] = " ".join(example['dialog'])
    return example

dataset = dataset.map(concatenate_utterances)

Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [30]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-medium')

In [31]:
def encode(examples):
    encoded = tokenizer(examples['dialog'], truncation=True, padding='max_length', max_length=128)
    encoded['labels'] = encoded['input_ids'][:]
    return encoded

encoded_dataset = dataset.map(encode, batched=True)

Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [34]:
output_dir = "new_outputs"
training_args = TrainingArguments(
    output_dir=output_dir,
    logging_steps = 200,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
)


In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation']
)

In [36]:
trainer.train()

Step,Training Loss
200,2.008400
400,1.845700
600,1.783000
800,1.696200
1000,1.615600
1200,1.613600
1400,1.601100
1600,1.496200
1800,1.490800
2000,1.491600


TrainOutput(global_step=3475, training_loss=1.5420171592218412, metrics={'train_runtime': 5578.1179, 'train_samples_per_second': 9.966, 'train_steps_per_second': 0.623, 'total_flos': 1.290661790220288e+16, 'train_loss': 1.5420171592218412, 'epoch': 5.0})

In [37]:
trainer.save_model("./daily_dialog_model")
tokenizer.save_pretrained('./daily_dialog_tokenizer')

('./daily_dialog_tokenizer/tokenizer_config.json',
 './daily_dialog_tokenizer/special_tokens_map.json',
 './daily_dialog_tokenizer/vocab.json',
 './daily_dialog_tokenizer/merges.txt',
 './daily_dialog_tokenizer/added_tokens.json',
 './daily_dialog_tokenizer/tokenizer.json')

In [133]:
my_model = AutoModelForCausalLM.from_pretrained("/content/daily_dialog_model")
my_tokenizer = AutoTokenizer.from_pretrained('/content/daily_dialog_tokenizer')

In [135]:
for step in range(4):
    new_user_input_ids = my_tokenizer.encode(my_tokenizer.eos_token + input(">> User:"), return_tensors='pt')

    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=bot_input_ids.dim()-1) if step > 0 else new_user_input_ids

    chat_history_ids = my_model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    print("DialoGPT: {}".format(my_tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[1]:][0], skip_special_tokens=True)))


>> User:HI
DialoGPT: , I'm looking for a good gift for my daughter.   What kind of gift do you want?   I want a necklace.   What kind of necklace?   A diamond necklace.   How about this one?   It's beautiful.   How much is it?   It's only $ 200.   That's a lot of money.   It's a lot of money for a necklace.   It's a lot of money for a necklace.   I want to buy a necklace too.   What kind of necklace do you want?   I  How about this one?    It's beautiful.    I'll take it.    OK. 
>> User:I can help you
DialoGPT:  with the shopping?    OK.   OK.   Thank you. 
>> User:what do you want to buy?
DialoGPT:       OK.    Thank you.     Thank you. 
>> User:Not at all
DialoGPT: . 


In [49]:
data = pd.read_csv('/content/dataset.csv')

In [50]:
data.head()

,question,answer
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.


Можно заметить, что dataset по своей сути содержит все ответы в столбце questions. То есть на кажду реплику тут есть реплика.

Сначла обучу изначальную модель на этом datatsete,  а потом свою

In [51]:
def concatenate_utterances(example):
    example['question'] = " ".join(example['question'])
    return example

In [53]:
def encode(examples):
    encoded = tokenizer(examples['question'], truncation=True, padding='max_length', max_length=128)
    encoded['labels'] = encoded['input_ids'][:]
    return encoded

In [59]:
train, valid = train_test_split(data, shuffle=True, test_size=0.2,  random_state=16)

In [60]:
data_dict = train.to_dict(orient='list')
data_train = Dataset.from_dict(train)
data_eval = valid.to_dict(orient='list')
data_eval = Dataset.from_dict(valid)

In [61]:
train = data_train.map(concatenate_utterances)
valid = data_eval.map(concatenate_utterances)

Map:   0%|          | 0/2980 [00:00<?, ? examples/s]

Map:   0%|          | 0/745 [00:00<?, ? examples/s]

In [65]:
train = train.map(encode)
valid = valid.map(encode)

Map:   0%|          | 0/2980 [00:00<?, ? examples/s]

Map:   0%|          | 0/745 [00:00<?, ? examples/s]

In [66]:
output_dir = "outputs_from_model"
training_args = TrainingArguments(
    output_dir=output_dir,
    logging_steps = 200,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
)

In [67]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=valid
)

In [68]:
trainer.train()

Step,Training Loss
200,0.467100
400,0.365100
600,0.329300
800,0.310800


TrainOutput(global_step=935, training_loss=0.35833344485033003, metrics={'train_runtime': 1468.7335, 'train_samples_per_second': 10.145, 'train_steps_per_second': 0.637, 'total_flos': 3459410087116800.0, 'train_loss': 0.35833344485033003, 'epoch': 5.0})

Мой dataset чище первого, на котором обучала, потому что в нем в каждой строке отдельное предложение. Поэтому показатели loss у него лучше. В dialog_dataset в одной строке могло быть по несколько похожих, но не связанных предложений, поэтоу chat выдает старнные ответы.

In [69]:
trainer.save_model("./base_model")
tokenizer.save_pretrained('./base_tokenizer')

('./base_tokenizer/tokenizer_config.json',
 './base_tokenizer/special_tokens_map.json',
 './base_tokenizer/vocab.json',
 './base_tokenizer/merges.txt',
 './base_tokenizer/added_tokens.json',
 './base_tokenizer/tokenizer.json')

In [75]:
mod_model = AutoModelForCausalLM.from_pretrained("/content/base_model")
mod_tokenizer = AutoTokenizer.from_pretrained('/content/base_tokenizer')

In [113]:
for step in range(4):

    new_user_input_ids = mod_tokenizer.encode(tokenizer.eos_token + input(">> You:"), return_tensors='pt')

    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=bot_input_ids.dim()-1) if step > 0 else new_user_input_ids

    chat_history_ids = mod_model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    print("Boddy: {}".format(mod_tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[1]:][0], skip_special_tokens=True)))

>> You:what's up
Boddy: ?   i'm   t r u e.   i   w a n t   t o   g o   t o   t h e   b e a c h   w o r l d   t o   s e e   y o u   t h i n k   i t.
>> You:I want to go with you
Boddy: .   i'm   a w a n t e d.
>> You:Do you like lions?
Boddy:    i'm afraid i'm a w a n t
>> You:I need to go
Boddy: .   


Моя репрезентация гибрида - не переписыванием архитектуры трансформера, а обучаю уже обученную мной на dialog_datase модель на новом dataset-e

In [114]:
output_dir = "outputs_from_my_model"
training_args = TrainingArguments(
    output_dir=output_dir,
    logging_steps = 200,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
)

In [115]:
trainer = Trainer(
    model=my_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=valid
)

In [117]:
trainer.train()

Step,Training Loss
200,0.467100
400,0.365100
600,0.329300
800,0.310800


TrainOutput(global_step=935, training_loss=0.35833344485033003, metrics={'train_runtime': 1614.8825, 'train_samples_per_second': 9.227, 'train_steps_per_second': 0.579, 'total_flos': 3459410087116800.0, 'train_loss': 0.35833344485033003, 'epoch': 5.0})

In [122]:
trainer.save_model("./with_my_model")
tokenizer.save_pretrained('./with_my_model')

('./with_my_model/tokenizer_config.json',
 './with_my_model/special_tokens_map.json',
 './with_my_model/vocab.json',
 './with_my_model/merges.txt',
 './with_my_model/added_tokens.json',
 './with_my_model/tokenizer.json')

In [125]:
wm_model = AutoModelForCausalLM.from_pretrained("/content/with_my_model")
wm_tokenizer = AutoTokenizer.from_pretrained('/content/with_my_model')

In [129]:
for step in range(1):

  new_user_input_ids = wm_tokenizer.encode(wm_tokenizer.eos_token + input(">> You:"), return_tensors='pt')

  bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=bot_input_ids.dim()-1) if step > 0 else new_user_input_ids

  chat_history_ids = wm_model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

  print("Boddy: {}".format(wm_tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[1]:][0], skip_special_tokens=True)))

>> You:Hello
Boddy: , I'm   t h e r e   w o r r i d e   y o u   g o i n g   t o   d o   t h e   b e a c h   w o r l d   w a n t   t o   g o   t o   t h e   b e a c h   w o r l d.


Запуска на среде T4 GPU, но на диске осталось мало памяти так-как тестила 3 модели. Поэтому вывожу только по одиной реплике.